In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 8.5 MB/s eta 0:00:000:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 4.8 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.9 MB/s eta 0:00:00


# Imports

In [ ]:
import os
import tempfile

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
import torch
from torchvision.io import read_video
import torch.nn as nn
import cv2
from pathlib import Path
from tqdm import tqdm
from models.experimental import attempt_load
from utils.general import non_max_suppression

tmpdir = tempfile.mkdtemp()

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)


2023-12-04 23:51:59.597084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 23:51:59.661889: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 23:51:59.662033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 2.5 MB/s eta 0:00:003.3 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 4.8 MB/s eta 0:00:00m eta 0:00:010:01:01m
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=f16898be5440dfa0ef162275f3bcf666936eebc034265f2605a6bc0be45d4cb3
  Stored in directory: /home/iiitb/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=c852c639c69591f9ce00be850c08d622447156ecb5d6607d71a46e663fb19f5a
  Stored in directory: /home/iiitb/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e4

In [ ]:
# Define the YOLOv8 model and load weights
model =  YOLO('yolov8n.pt')
stride = int(model.stride.max())  # model stride

In [ ]:
# Set threshold for confidence score and non-maximum suppression
conf_threshold = 0.5
iou_threshold = 0.5

# Initialize the video capture
video_path = 'path/to/your/video/file.mp4'
cap = cv2.VideoCapture(video_path)

# Get video details
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize output video writer
output_path = 'output.avi'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width, frame_height))

# Initialize an empty list to store detected objects' coordinates
detected_objects = []

# Process each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform YOLOv8 inference
    result = model(rgb_frame, size=stride)
    pred = non_max_suppression(result, conf_threshold, iou_threshold)[0]

    # Draw bounding boxes on the frame
    if pred is not None:
        for det in pred:
            x1, y1, x2, y2, conf, cls = det
            if int(cls) == 0:  # 'person' class index
                detected_objects.append([x1, y1, x2, y2])
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Write the frame to the output video
    out.write(frame)

# Release video capture and writer
cap.release()
out.release()

In [ ]:
# Convert the detected_objects list to a NumPy array
detected_objects_array = np.array(detected_objects)

# Display or save the NumPy array as needed
print(detected_objects_array)

Frame selector algorithm and joint location and action localisation

In [ ]:
import tensorflow as tf
import cv2 as cv

MODEL_PATH = 'model/path/here'
movenet_model = tf.lite.Interpreter(model_path=MODEL_PATH)
movenet_model.allocate_tensors()


def get_movenet_data(path):
    estimator = tf.lite.Interpreter(model_path='/content/drive/MyDrive/lite-model_movenet_singlepose_thunder_3.tflite')
    #estimator = tf.lite.Interpreter(model_path='/content/drive/MyDrive/lite-model_mediapipe_3.tflite')
    estimator.allocate_tensors()
    
    cap = cv.VideoCapture(path)
    if not cap.isOpened():
        print("Error: Unable to open video.")
        return [], []

    frames = []
    vid_keypts = []

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Preprocess frame for MoveNet model
            img1 = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), 256, 341)
            frames.append(np.squeeze(img1))

            # Preprocess frame for keypoint estimation
            img = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), 256, 256)
            input_image = tf.cast(img, dtype=tf.float32)

            # Model inference
            input_details = estimator.get_input_details()
            output_details = estimator.get_output_details()
            estimator.set_tensor(input_details[0]['index'], np.array(input_image))
            estimator.invoke()
            keypts = estimator.get_tensor(output_details[0]['index'])
            vid_keypts.append(keypts)

        else:
            break

    cap.release()
    return vid_keypts, frames


def frame_with_max_change(keypts):
    max_diff = 0
    max_loc = 0

    for frame in range(len(keypts) - 1):
        shaped1 = np.squeeze(keypts[frame])
        shaped2 = np.squeeze(keypts[frame + 1])

        # Calculate the Euclidean distance between consecutive frames
        diff_mag = np.linalg.norm(shaped2 - shaped1)

        if max_diff < diff_mag:
            max_diff = diff_mag
            max_loc = frame

    return max_diff, max_loc


def processed(keypts):
    # Extract x and y coordinates and flatten for each frame
    return [np.squeeze(k)[:, :2].flatten() for k in keypts]


def get_best_frames(video_dir, test=False):
    best_frames = []
    all_keypts = []
    y = []

    # Set Label
    set_label = np.array([1.0, 0.0, 0.0], dtype=np.float32)
    if 'headbanging' in video_dir and not test:
        set_label = np.array([0.0, 1.0, 0.0], dtype=np.float32)
    elif 'spinning' in video_dir and not test:
        print("SPINNING -- TRAIN")
        set_label = np.array([0.0, 0.0, 1.0], dtype=np.float32)

    for video in os.listdir(video_dir):
        if 'noclass' in video:
            continue

        keypts, frames = get_movenet_data(os.path.join(video_dir, video))
        all_keypts.append(processed(keypts))

        _, max_loc = frame_with_max_change(keypts)
        best_frames.append(frames[max_loc + 1])

        if not test:
            y.append(set_label)
        else:
            # Extract labels for test set
            if 'armflapping' in video:
                y.append(np.array([1.0, 0.0, 0.0], dtype=np.float32))
            elif 'headbanging' in video:
                y.append(np.array([0.0, 1.0, 0.0], dtype=np.float32))
            elif 'spinning' in video:
                y.append(np.array([0.0, 0.0, 1.0], dtype=np.float32))

    if test:
        test_keys, test_best_frames, test_labels = [], [], []
        for frame, keypt, label in zip(best_frames, all_keypts, y):
            if len(keypt) == 40:
                test_keys.append(keypt)
                test_best_frames.append(frame)
                test_labels.append(label)

        return (np.array(test_best_frames, dtype=np.float32), np.array(test_keys, dtype=np.float32)), np.array(
            test_labels, dtype=np.float32)

    return (best_frames, all_keypts), y


Main model which does classification of ASD and No ASD based on stimming actions detected

In [ ]:
class MultimodalSSBDModel(nn.Module):
    """
    Detects the presence and type of stimming behavior using video and Movenet features.

    Components:
        - Spatiotemporal convolutional block (extracts features from video)
        - Bidirectional LSTM with Multihead Attention (processes temporal dynamics)
        - Fully-connected network (classifies stimming type)

    Args:
        in_channels (int): Number of video input channels
        intermediate_channels (int): Intermediate channels in the spatiotemporal block
        out_channels (int): Output channels from the spatiotemporal block
        kernel_size (tuple): Kernel size for both spatial and temporal dimensions
        strides (tuple): Strides for both spatial and temporal dimensions
        lstm_dim_embedding (int): Dimension of LSTM hidden state
        lstm_dim_hidden (int): Dimension of LSTM hidden layer
        num_lstm_layers (int): Number of bidirectional LSTM layers
        fc_layer_1_dim (int): Dimension of the first fully-connected layer
        fc_layer_2_dim (int): Dimension of the second fully-connected layer
        n_classes (int): Number of stimming behavior classes
        dropout_rate (float): Dropout rate applied in LSTM and Attention block
        use_movenet (bool): Flag to use Movenet features (default True)
        n_frames (int): Number of video frames used as input (default 40)

    """

    def __init__(self, in_channels, intermediate_channels, out_channels,
                 kernel_size, strides, lstm_dim_embedding, lstm_dim_hidden,
                 num_lstm_layers, fc_layer_1_dim, fc_layer_2_dim, n_classes,
                 dropout_rate=0.2, use_movenet=True, n_frames=40):
        super().__init__()

        # Spatiotemporal feature extraction
        self.spatiotemporal_conv = nn.Sequential(
            nn.Conv3d(in_channels, intermediate_channels,
                      kernel_size=(1, *kernel_size), stride=(1, *strides), padding='valid'),
            nn.BatchNorm3d(intermediate_channels),
            nn.ReLU(),
            nn.Conv3d(intermediate_channels, out_channels,
                      kernel_size=(*kernel_size, 1), stride=(*strides, 1), padding='valid'),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(),
        )

        # Bidirectional LSTM with Multihead Attention
        self.lstm = nn.LSTM(
            lstm_dim_embedding + movenet_dim if use_movenet else lstm_dim_embedding,
            lstm_dim_hidden, num_layers=num_lstm_layers, dropout=dropout_rate, bidirectional=True)
        self.attn = nn.MultiheadAttention(embed_dim=lstm_dim_embedding, num_heads=1, dropout=dropout_rate / 4)

        # Fully-connected network for classification
        self.fc = nn.Sequential(
            nn.Linear(lstm_dim_hidden * 2, fc_layer_1_dim),
            nn.LeakyReLU(),
            nn.BatchNorm1d(fc_layer_1_dim),
            nn.Linear(fc_layer_1_dim, fc_layer_2_dim),
            nn.LeakyReLU(),
            nn.BatchNorm1d(fc_layer_2_dim),
            nn.Linear(fc_layer_2_dim, n_classes),
            nn.Softmax(dim=1),
        )

        self.use_movenet = use_movenet
        self.n_frames = n_frames

def forward(self, video, movenet_features=None):
    """
    Args:
        video (torch.Tensor): Video frames (shape: [batch_size, n_frames, channels, height, width])
        movenet_features (torch.Tensor): Movenet features (optional, shape: [batch_size, n_frames, movenet_dim])

    Returns:
        torch.Tensor: Output logits for stimming type classification (shape: [batch_size, n_classes])
    """

    hidden = None
    features_list = []

    for frame_idx in range(self.n_frames):
        # Extract features from video frame
        frame_features = self.spatiotemporal_conv(video[:, frame_idx, :, :, :])

        # Combine features with joint location from mediapipe 
        if self.use_movenet and movenet_features is not None:
            combined_features = torch.cat([frame_features, movenet_features[:, frame_idx, :]], dim=1)
        else:
            combined_features = frame_features

        features_list.append(combined_features.unsqueeze(0))  # Add dimension for LSTM

    # Pass features through LSTM and Attention
    lstm_out, hidden = self.lstm(torch.cat(features_list), hidden)
    attn_output, attn_weights = self.attn(lstm_out, lstm_out, lstm_out)

    # Use last hidden state for classification
    logits = self.fc(attn_output[:, -1, :])

    return logits




In [ ]:
def predict(video_frames, movenet_features, ssbd_model):
    """
    Classify ASD and No_ASD based on stimming action.

    Args:
        video_frames (torch.Tensor): Video frames (shape: [batch_size, n_frames, channels, height, width])
        movenet_features (torch.Tensor): Movenet features (shape: [batch_size, n_frames, movenet_dim])
        ssbd_model (MultimodalSSBDModel): An instance of the MultimodalSSBDModel

    Returns:
        str: "ASD" or "No_ASD" based on the classification result
    """

    # Ensure the model is in evaluation mode
    ssbd_model.eval()

    # Convert video frames and Movenet features to the appropriate device
    video_frames = video_frames.to(device=ssbd_model.device)  # TODO
    movenet_features = movenet_features.to(device=ssbd_model.device) # TODO

    with torch.no_grad():
        # Call the forward method of MultimodalSSBDModel
        logits = ssbd_model(video_frames, movenet_features)

        # Perform argmax to get the predicted class
        _, predicted_class = torch.max(logits, dim=1)

        # Map the predicted class to the corresponding label
        class_mapping = {0: "No_ASD", 1: "ASD"}
        predicted_label = class_mapping[predicted_class.item()]

    return predicted_label



